In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 654621943911384659
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 7460077222521619665
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 8556454351109010635
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11276946637
locality {
  bus_id: 1
  links {
  }
}
incarnation: 726840077489381896
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


In [24]:
# Imports
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM
from sklearn.model_selection import train_test_split
import time
from math import log10, floor

In [17]:
# Define variables
n_vars = 3
var_ids = list(range(n_vars))
var_names = ['var' + str(i) for i in var_ids]
var_weights = [0.1, 0.6, 0.3] # variable distribution of mock data
n_time_steps = 20
n_individuals = 1000

In [25]:
# Helper function(s)

# round a number to n significant digits
def round_to_n(x, n = 2):
    return round(x, -int(floor(log10(abs(x)))) + (n - 1)) if x != 0 else 0

In [29]:
# Generate mock data

data = np.zeros(shape=(n_individuals, n_time_steps, n_vars))

start_time = time.time()

for inv in range(n_individuals):
    for t in range(n_time_steps):
        var = np.random.choice(var_ids, p=var_weights)
        data[inv][t][var] = 1
        
print('time taken:', round_to_n(time.time() - start_time), 'seconds')
        
#print(data)

time taken: 0.53 seconds


In [30]:
np.sum(data, axis=0)

array([[ 82., 586., 332.],
       [ 95., 606., 299.],
       [ 91., 607., 302.],
       [104., 603., 293.],
       [107., 608., 285.],
       [ 87., 618., 295.],
       [100., 588., 312.],
       [106., 569., 325.],
       [ 88., 618., 294.],
       [ 99., 583., 318.],
       [121., 592., 287.],
       [105., 592., 303.],
       [101., 593., 306.],
       [ 98., 623., 279.],
       [118., 580., 302.],
       [ 90., 630., 280.],
       [102., 609., 289.],
       [ 82., 613., 305.],
       [ 98., 610., 292.],
       [113., 569., 318.]])

In [28]:
# Divide the data into train and test sets 

data_train, data_test = train_test_split(data, test_size = 0.2)
print(data_train.shape, data_test.shape)

(800, 20, 3) (200, 20, 3)
